#### Rose Hedderman and Nickolas Gibson
##### EID: rrh2298 and ndg644
## Project 5

##### Get shopify data from zip file

In [4]:
%env PGINSTANCE=pg

env: PGINSTANCE=pg


In [5]:
%env BASE_DIR=/home/jupyter/rose/snippets

env: BASE_DIR=/home/jupyter/rose/snippets


In [6]:
!gsutil cp gs://cs327e-open-access/shopify.zip $BASE_DIR

Copying gs://cs327e-open-access/shopify.zip...
/ [1 files][  3.2 MiB/  3.2 MiB]                                                
Operation completed over 1 objects/3.2 MiB.                                      


In [7]:
!unzip $BASE_DIR/shopify.zip

Archive:  /home/jupyter/rose/snippets/shopify.zip
replace __MACOSX/._shopify? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


#### See ERD: shopify-firestore-erd.pdf

#### Create the Firestore collections and subcollections based on your design and populate them with the Shopify records.

In [9]:
# Reviews Collection
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/reviews.csv', sep=',', header=0, lineterminator='\n')
rows = df.values.tolist()

frontSlash = '/'
backSlash = '\\'

reviewsCount = 0
for row in rows: 
    reviewsCount += 1
    record = {}
    record['app_id'] = row[0]
    if frontSlash or backSlash not in row[1]:
        record['author'] = row[1]
    record['rating'] = row[2]
    record['posted_at'] = row[3]
    
    class_ref = db.collection('reviews').document(row[0])
    batch.set(class_ref, record)
    
    if reviewsCount%500 == 0:
        batch.commit()
        
batch.commit()

[update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_time {
   seconds: 1649177006
   nanos: 782542000
 },
 update_

In [13]:
# Categories Collection
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/categories.csv', sep=',', header=0, lineterminator='\n')
cat_rows = df.values.tolist()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

df = pd.read_csv('shopify/apps_categories.csv', sep=',', header=0, lineterminator='\n')
appCat_rows = df.values.tolist()

# keep track of items in the batch
batchCount = 0
# count number of items in Categories Collection
catCount = 0
# count number of items in Apps SubCollection
appSubCount = 0

for cat_row in cat_rows:
    cat_record = {}
    cat_record['id'] = cat_row[0]
    cat_record['title'] = cat_row[1]
    
    cat_ref = db.collection('categories').document(cat_row[0])
    catCount += 1
    batch.set(cat_ref, cat_record)
    batchCount += 1
    if batchCount == 500:
        batch.commit()
        batchCount = 0
    
    for appCat_row in appCat_rows:
        appCat_record = {}
        appCat_record['app_id'] = appCat_row[0]
        appCat_record['category_id'] = appCat_row[1]
        
        if cat_row[0] == appCat_row[1]:
            
            for apps_row in app_rows:
            #const junctionRef = db.doc(`junction_student_course/${studentId}_${courseId}`);
            #await junctionRef.set({ studentId, courseId });
                
                apps_record = {}
                apps_record['id'] = apps_row[0]
                apps_record['url'] = apps_row[1]
                apps_record['title'] = apps_row[2]
                apps_record['developer'] = apps_row[3]
                apps_record['developer_link'] = apps_row[4]
                apps_record['icon'] = apps_row[5]
                apps_record['rating'] = apps_row[6]
                apps_record['reviews_count'] = apps_row[7]
                
                if appCat_row[0] == apps_row[0]:
                    junctionref = db.collection('categories').document(cat_row[0]).collection('apps').document(apps_row[0])
                    appSubCount += 1
                    batch.set(junctionref, apps_record)
                    batchCount += 1
                    if batchCount == 500:
                        batch.commit()
                        batchCount = 0


In [15]:
# Apps Collection with Key_benefits subcollection
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/apps.csv', sep=',', header=0, lineterminator='\n')
app_rows = df.values.tolist()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

# keep track of items in the batch
batchCount = 0
# count number of items in Apps Collection
appCount = 0
# count number of items in key_benefits subcollection
kbCount = 0

for app_row in app_rows:
    app_record = {}
    app_record['id'] = app_row[0]
    app_record['url'] = app_row[1]
    app_record['title'] = app_row[2]
    app_record['developer'] = app_row[3]
    app_record['developer_link'] = app_row[4]
    app_record['icon'] = app_row[5]
    app_record['rating'] = app_row[6]
    app_record['reviews_count'] = app_row[7]
    
    app_ref = db.collection('apps').document(app_row[0])
    appCount += 1
    batch.set(app_ref, app_record)
    batchCount += 1
    
    if batchCount == 500:
        batch.commit()
        batchCount = 0
        
    # make key benefits subcollection
    for kb_row in kb_rows:
        kb_record = {}
        kb_record['app_id'] = kb_row[0]
        kb_record['title'] = kb_row[1]
        kb_record['description'] = kb_row[2]
        
        if app_row[0] == kb_row[0]:
            kb_ref = db.collection('apps').document(app_row[0]).collection('key_benefits').document(kb_row[0])
            kbCount += 1
            batch.set(kb_ref, kb_record)
            batchCount += 1
            if batchCount == 500:
                batch.commit()
                batchCount = 0
    batch.commit()
batch.commit()


[]

In [27]:
# Apps Collection with pricing plan subcollection
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
batch = db.batch()

df = pd.read_csv('shopify/key_benefits.csv', sep=',', header=0, lineterminator='\n')
kb_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plans.csv', sep=',', header=0, lineterminator='\n')
pp_rows = df.values.tolist()

df = pd.read_csv('shopify/pricing_plan_features.csv', sep=',', header=0, lineterminator='\n')
ppf_rows = df.values.tolist()

# keep track of items in the batch
batchCount = 0
# count number of items in pricing_plans subcollection
ppCount = 0

# make pricing plans subcollection
for ppf_row in ppf_rows:
    ppf_record = {}
    ppf_record['app_id'] = ppf_row[0]
    ppf_record['pricing_plan_id'] = ppf_row[1]
    ppf_record['feature'] = ppf_row[2]
        
    for app_row in app_rows:
        if app_row[0] == ppf_row[0]:
            for pp_row in pp_rows:
                pp_record = {}
                pp_record['id'] = pp_row[0]
                pp_record['app_id'] = pp_row[1]
                if app_row[0] == pp_row[1]:
                    pp_record['title'] = pp_row[2]
                    pp_record['description'] = pp_row[3]
                    pp_record['pricing_plan_features'] = ppf_row[2]
                    pp_ref = db.collection('apps').document(app_row[0]).collection('pricing_plans').document(pp_row[0])
                    ppCount += 1
                    batch.set(pp_ref, pp_record)
                    batchCount += 1
                    if batchCount == 500:
                        batch.commit()
                        batchCount = 0
        batch.commit()
        batchCount += 1
        if batchCount == 500:
            batch.commit()
            batchCount = 0
    
    batch.commit()
    batchCount += 1
    if batchCount == 500:
        batch.commit()
        batchCount = 0
batch.commit()

[]

#### Get the number of documents in each collection and subcollection

In [24]:
print("There are",reviewsCount,"documents in the Reviews collection.")
print("There are",catCount,"documents in the Category collection.")
print("There are",appSubCount,"documents in the Apps subcollection.")
print("There are",appCount,"documents in the Apps collection.")
print("There are",kbCount,"documents in the Key Benefits subcollection.")
print("We were unable to find a way to count the pricing plans subcollection. We made the mistake of not counting them when the subcollection was first made, and rerunning it takes a very long time. There should be", len(pp_rows),"documents in the pricing plans subcollection.")

There are 124601 documents in the Reviews collection.
There are 12 documents in the Category collection.
There are 5383 documents in the Apps subcollection.
There are 3547 documents in the Apps collection.
There are 9541 documents in the Key Benefits subcollection.
We were unable to find a way to count the pricing plans subcollection. We made the mistake of not counting them when the subcollection was first made, and rerunning it takes a very long time. There should be 6275 documents in the pricing plans subcollection.


#### List the top 10 "Productivity" apps (whose categories.title = "Productivity") sorted by their rating in descending order. Return the id, title, developer, rating and
#### reviews_count for those apps. Limit the results to the first 10 records. 
##### Note: this query refers to access pattern #1

In [25]:
db = firestore.Client()
category_ref = db.collection('categories')
query = category_ref.where('title', '==', 'Productivity')
results = query.stream()

for result in results:
    
    aresults = category_ref.document(result.id).collection('apps').order_by(u'rating',direction =firestore.Query.DESCENDING).limit(10).stream()
    
    for aresult in aresults:
        final = aresult.to_dict()
        print(final['id'], final['title'], final['developer'], final['rating'], final['reviews_count'])

fe5ae45c-379c-42bf-be7d-b5f6e15fc13c Customer Tags Union Works Apps 5.0 21
f99bb1e3-f326-4f10-8901-491652e9809b Order Tagger Union Works Apps 5.0 68
f864e3bd-da0e-41dc-be65-984325331475 SilkRoad ‑ Facebook Auto Ads SilkRoad 5.0 2
f5344e64-9cda-4d97-b198-2aaeb5170518 Xporter Data Export Tool Modd Apps Inc. 5.0 223
ed77a32d-0fa3-458b-b639-e01ea7b78ec0 FraudBlock Fraud Prevention ShopFox 5.0 6
e0e231d6-4988-4a8c-ad84-cf18d0f38738 Automation Fox Automation Fox 5.0 1
d75fa395-3a4d-41c3-88d1-2f802c1e2411 AirPower BaseGenius 5.0 1
d0be03ff-f74d-4fb5-8d2c-79541a1aea1b Excelify Excelify.io 5.0 178
c9a86f8c-817e-4bb5-bbdd-e0e669bd7816 UPC Code Manager Ventures & Adventures 5.0 1
c7578030-19fd-42e1-a5a5-03c7861220aa EzySlips ‑ Shipping Automation Onjection Labs 5.0 219


#### List the 10 apps with the highest number of reviews (based on apps.review_count). Return the id, title, developer, rating and reviews_count for those apps.
#### Order the results by reviews_count in descending order.
##### Note: this is query refers to access pattern #2.

In [26]:
import pandas as pd
from google.cloud import firestore
db = firestore.Client()
app_ref = db.collection(u'apps')
query = app_ref.order_by(u'reviews_count',direction =firestore.Query.DESCENDING).limit(10)
results = query.stream()

for result in results:
    aresults = result.to_dict()
    print(aresults['id']+', '+aresults['title']+', '+aresults['developer']+',',aresults['rating'],',',aresults['reviews_count'])

d9f142ee-b141-4dc4-9353-173db61d2eb0, Privy ‑ Exit Pop Ups & Email, Privy, 4.7 , 23078
78ea0810-c008-4a4e-a82f-de0c790e3286, Free Shipping Bar, Hextom, 4.9 , 8737
b88488b0-9912-44d3-b736-224c36f09d95, Sales Pop ‑ Popup Notification, CartKit, 4.8 , 6905
e528a60e-94f8-4e92-80e2-5bc6013b8283, BEST Currency Converter, Grizzly Apps, 4.8 , 5986
be2640c4-01b5-4d52-9f68-cae8c0734d0d, Recart FB Messenger Marketing, Recart, 4.8 , 5596
70bff9e0-4316-4cc6-84ce-92fcd1bc6925, EU Cookie Bar ‑ Cookie GDPR, Booster Apps, 4.7 , 5259
171816e2-27d4-4552-a65e-ab44a312fe04, Sales Pop Master ‑ Countdown, Autoketing, 4.8 , 4931
9025eff0-d714-4df1-930f-43f5582979ad, Ultimate Sales Boost, Hextom, 4.8 , 4847
404183ac-6d52-4960-85d2-8350f37c2f0b, Ads by Varinode, Varinode, Inc., 4.9 , 4245
0380c5ed-7b91-4d4b-9a4e-f6a1c50269dd, Quick Announcement Bar, Hextom, 4.9 , 4237
